In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn import metrics
import re
from sklearn import tree
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
from collections import OrderedDict
!pip install XlsxWriter
import xlsxwriter
from subprocess import call
import csv
import openpyxl

In [ ]:
csvFileName = 'errol_mclaughlin_jr__10001__DKR_400bp_11-19-01.csv'
excelName = 'errol_mclaughlin_jr__10001__DKR_400bp_11-19-01.xlsx'
df = pd.read_csv(csvFileName, error_bad_lines=False)
df.head()

,Index,File,Sheet,cell_address,isNumeric,isFormula,length_of_cell,number_of_words,leading_spaces,first_character_number,first_character_special,capitalized,upper_cased,contain_alpha_numeric,contain_special_characters,contain_punctuation,contain_colon,contain_total,contain_table,in_year_range,aggr_formula_used,reference_formula_value_numeric,first_row_number,first_column_number,neighbors=0,neighbors=1,neighbors=2,neighbors=3,neighbors=4,h_alignment=left,h_alignment=center,h_alignment=right,h_alignment=default,v_alignment=top,v_alignment=center,v_alignment=bottom,indendation,fill_pattern=default,is_text_wrapped,cell_size,no_top_border,thin_top_border,no_bottom_border,no_left_border,no_right_border,medium_right_border,no_of_borders,font_size,is_default_font_color,is_bold,is_single_underlined
0,1,errol_mclaughlin_jr__10001__DKR_400bp_11-19-01...,GasDailyVal,A1,0,0,9,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
1,2,errol_mclaughlin_jr__10001__DKR_400bp_11-19-01...,GasDailyVal,B1,0,0,6,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
2,3,errol_mclaughlin_jr__10001__DKR_400bp_11-19-01...,GasDailyVal,C1,0,0,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
3,4,errol_mclaughlin_jr__10001__DKR_400bp_11-19-01...,GasDailyVal,E1,0,0,10,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
4,5,errol_mclaughlin_jr__10001__DKR_400bp_11-19-01...,GasDailyVal,F1,0,0,8,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0


In [ ]:
df.columns

Index(['Index', 'File', 'Sheet', 'cell_address', 'isNumeric', 'isFormula',
       'length_of_cell', 'number_of_words', 'leading_spaces',
       'first_character_number', 'first_character_special', 'capitalized',
       'upper_cased', 'contain_alpha_numeric', 'contain_special_characters',
       'contain_punctuation', 'contain_colon', 'contain_total',
       'contain_table', 'in_year_range', 'aggr_formula_used',
       'reference_formula_value_numeric', 'first_row_number',
       'first_column_number', 'neighbors=0', 'neighbors=1', 'neighbors=2',
       'neighbors=3', 'neighbors=4', 'h_alignment=left', 'h_alignment=center',
       'h_alignment=right', 'h_alignment=default', 'v_alignment=top',
       'v_alignment=center', 'v_alignment=bottom', 'indendation',
       'fill_pattern=default', 'is_text_wrapped', 'cell_size', 'no_top_border',
       'thin_top_border', 'no_bottom_border', 'no_left_border',
       'no_right_border', 'medium_right_border', 'no_of_borders', 'font_size',
       'is

In [ ]:
address = df["cell_address"]
sheetIndexes = df["Sheet"]

In [ ]:
sheets = list(set(df["Sheet"]))
sheets

['AECOApr-Oct',
 'RockiesApr-Oct',
 'RockiesDec-Mar',
 'Cal02_Cal03',
 'AIG Weather',
 'GasDailyVal',
 'DiscountRate',
 'Nymex',
 'Totals',
 'AIG Crude & Products']

In [ ]:
columns_to_drop = ["Index", "File", "Sheet", "cell_address"]
df = df.drop(columns_to_drop, axis=1)
df.head()

,isNumeric,isFormula,length_of_cell,number_of_words,leading_spaces,first_character_number,first_character_special,capitalized,upper_cased,contain_alpha_numeric,contain_special_characters,contain_punctuation,contain_colon,contain_total,contain_table,in_year_range,aggr_formula_used,reference_formula_value_numeric,first_row_number,first_column_number,neighbors=0,neighbors=1,neighbors=2,neighbors=3,neighbors=4,h_alignment=left,h_alignment=center,h_alignment=right,h_alignment=default,v_alignment=top,v_alignment=center,v_alignment=bottom,indendation,fill_pattern=default,is_text_wrapped,cell_size,no_top_border,thin_top_border,no_bottom_border,no_left_border,no_right_border,medium_right_border,no_of_borders,font_size,is_default_font_color,is_bold,is_single_underlined
0,0,0,9,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
1,0,0,6,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
2,0,0,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
3,0,0,10,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
4,0,0,8,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0


In [ ]:
import joblib

In [ ]:
loaded_model = joblib.load("finalized_model_full.sav")

In [ ]:
df.head()

,isNumeric,isFormula,length_of_cell,number_of_words,leading_spaces,first_character_number,first_character_special,capitalized,upper_cased,contain_alpha_numeric,contain_special_characters,contain_punctuation,contain_colon,contain_total,contain_table,in_year_range,aggr_formula_used,reference_formula_value_numeric,first_row_number,first_column_number,neighbors=0,neighbors=1,neighbors=2,neighbors=3,neighbors=4,h_alignment=left,h_alignment=center,h_alignment=right,h_alignment=default,v_alignment=top,v_alignment=center,v_alignment=bottom,indendation,fill_pattern=default,is_text_wrapped,cell_size,no_top_border,thin_top_border,no_bottom_border,no_left_border,no_right_border,medium_right_border,no_of_borders,font_size,is_default_font_color,is_bold,is_single_underlined
0,0,0,9,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
1,0,0,6,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
2,0,0,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
3,0,0,10,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0
4,0,0,8,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,10,1,1,0


In [ ]:
arr = loaded_model.predict(df)
arr

array(['header', 'header', 'header', ..., 'data', 'data', 'data'],
      dtype=object)

In [ ]:
arr[2]

'header'

In [ ]:
from openpyxl import load_workbook
sheetNamePrefix = 'pred_with_full_'
wb = load_workbook(filename=excelName)
worksheets = {}
for i in range(len(sheets)):
  try:
    sheetName = sheetNamePrefix + sheets[i] 
    ws = wb[sheetName]
    wb.remove(ws)
    wb.save(excelName)
  except KeyError:
    #do nothing
    print("err")
  wb.create_sheet(sheetName)
  worksheets[sheets[i]] = wb[sheetName]
for i in range(len(address)):
  sheetName = sheetIndexes[i]
  worksheets[sheetName][address[i]] = arr[i]
wb.save(excelName)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
wb = load_workbook(filename = excelName)
for sheetName in sheets:
  sheetName = sheetNamePrefix + sheetName
  ws = wb[sheetName]
  for row in ws.iter_rows():
    headerCount = 0
    dataCount = 0
    emptyCells = 0
    for cell in row:
      # print(cell.coordinate)
      cellVal = str(cell.value)
      if cellVal == "None":
        emptyCells+=1
      elif cellVal == "data":
        dataCount+=1
      elif cellVal == "header":
        headerCount+=1
    print(len(row))
    print(emptyCells)
    print(headerCount)
    print(dataCount)

    cellsWithValue = len(row) - emptyCells
    if cellsWithValue > 0:
      if headerCount > dataCount:
        #header
        for cell in row:
          if str(cell.value) == "data":
            ws[cell.coordinate] = 'header'
      else:
        #data
        for cell in row:
          if str(cell.value) == "header":
            ws[cell.coordinate] = 'data'

wb.save(excelName)